# Tweepy

[Tweepy](http://tweepy.readthedocs.io/en/v3.5.0/)  
[marcobonzanini.com](https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/)

In [ ]:
# !pip install tweepy

In [ ]:
# %load ../_data/standard_import.txt

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
import twitter
import json

plt.style.use('seaborn-white')

In [ ]:
pickle_file = '../_credentials/twitter_credentials.pkl'
Twitter=pickle.load(open(pickle_file, 'rb'))

#### Authorizing an application to access Twitter account data

In [ ]:
import tweepy
from tweepy import OAuthHandler
 
consumer_key = 'YOUR-CONSUMER-KEY'
consumer_secret = 'YOUR-CONSUMER-SECRET'
access_token = 'YOUR-ACCESS-TOKEN'
access_secret = 'YOUR-ACCESS-SECRET'
 
auth = OAuthHandler(Twitter['Consumer Key'], Twitter['Consumer Secret'])
auth.set_access_token(Twitter['Access Token'], Twitter['Access Token Secret'])
 
api = tweepy.API(auth)

In [ ]:
api.wait_on_rate_limit = True
api.wait_on_rate_limit_notify = True

## Get current address and lat, long with HERE api

In [ ]:
import json
import os
import platform
import re
import ssl
import subprocess
import requests
import urllib.request
from urllib.request import urlretrieve
import plistlib
import tweepy

from PIL import Image

In [ ]:
import datetime
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

In [ ]:
import os, pickle

pickle_file='../_credentials/here_credentials.pkl'
Here=pickle.load(open(pickle_file,'rb'))

### Search address

In [ ]:
def geo_latlon_address(address):
    """Lookup address details as JSON using HERE api
    Decode UTF-8 bytes to Unicode and convert single quotes to double quotes to make it valid JSON.
    Load the JSON to a Python list & dump it back out as formatted JSON."""
    import os, pickle, requests, json
    
    Here=pickle.load(open('../_credentials/here_credentials.pkl','rb'))
    api_url = 'https://geocoder.cit.api.here.com/6.2/geocode.json'
    http = '{}?app_id={}&app_code={}&searchtext={}'.format(api_url, Here['app_id'], Here['app_code'], address)
    content = requests.get(http).content
    my_json = content.decode('utf8').replace("'", '"')
    data = json.loads(my_json)
    latlon_dict = data['Response']['View'][0]['Result'][0]['Location']['NavigationPosition'][0]
    return latlon_dict['Latitude'], latlon_dict['Longitude']

In [ ]:
geo_latlon_address('9, dorpsstraat, baambrugge')

In [ ]:
def payload():
    """Returns payload for POST request HERE api."""
    import re, ast
    
    hotspots = !airport scan
    pos_bssid = re.search(r'\b(BSSID)\b', hotspots[0]).start()
    bssid = {b[pos_bssid:pos_bssid+17]:b[pos_bssid+18:pos_bssid+21] for b in hotspots[1:]}
    payload = {} 
    payload['wlan'] = [ast.literal_eval('{}"mac": "{}", "powrx": {}{}'.format('{', b, int(r), '}')) for b, r in bssid.items()]
    return payload

In [ ]:
def geo_latlon():
    """Lookup address details as JSON using HERE api
    Decode UTF-8 bytes to Unicode and convert single quotes to double quotes to make it valid JSON.
    Load the JSON to a Python list & dump it back out as formatted JSON."""
    import requests, json
    
    api_url = 'https://pos.cit.api.here.com/positioning/v1/locate'
    http = '{}?app_id={}&app_code={}'.format(api_url, Here['app_id'], Here['app_code'])
    headers = {'Content-Type': 'application/json'}
    
    r = requests.post(http, data=json.dumps(payload()), headers=headers)
    print(r.status_code, r.reason)
    my_json = r.content.decode('utf8').replace("'", '"')
    
    return json.loads(my_json)['location']

In [ ]:
geo_latlon()

In [ ]:
def geo_address():
    """Reverse get address from latlon."""
    
    import ssl, urllib.request, json
    
    geo_latlonrad = geo_latlon()
    lat, lon, radius = geo_latlonrad['lat'], geo_latlonrad['lng'], geo_latlonrad['accuracy']
    
    context = ssl._create_unverified_context()
    rev_geocoder_url = 'https://reverse.geocoder.api.here.com/6.2/reversegeocode.json?prox='
    
    results = json.load(urllib.request.urlopen(
        '{}{},{},1000&mode=retrieveAddresses&maxResults=3&gen=8&app_id={}&app_code={}'.format(
            rev_geocoder_url, lat, lon, Here['app_id'], Here['app_code']), context=context)
                       )['Response']['View'][0]['Result']
    
    for result in results:
        if result['MatchQuality'].get('Street'):
            address = result['Location']['Address']['Label']
            break
        else:
            continue

    return address
geo_address()

In [ ]:
latlon = geo_latlon()

### Local topics by lat lon

In [ ]:
def local_search(keyword, radius, lang='nl', count=100):
    """Get tweets by topic within radius from current location
    http://docs.tweepy.org/en/v3.5.0/api.html#status-methods"""
    import datetime
    tweet_list = []
    geocode = '{},{},{}km'.format(latlon['lat'], latlon['lng'], radius)
    tweets = tweepy.Cursor(api.search, q=keyword, lang=lang, geocode=geocode, since=str(yesterday)).items(count)
    for tweet in tweets:
        try:
            tweet_list.append([tweet.geo['coordinates'], tweet.text])
        except:
            continue
    # dir(tweet): tweet.created_at, tweet.user.id , tweet.user.followers_count
    return tweet_list

In [ ]:
def get_lat_lon(address):
    """Return lat, lon, accuracy based on address(country, place, street)"""
    from geopy.geocoders import Nominatim
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    return {'lat': location.latitude, 'lng': location.longitude,'accuracy': 0.}


In [ ]:
keyword = 'japan'
radius = 20
lang = 'nl'
# loc_search = local_search(keyword, radius, lang)

In [ ]:
geocode = '{},{},{}km'.format(latlon['lat'], latlon['lng'], radius)
tweet_latlon = []
for t in tweepy.Cursor(api.search, q=keyword, lang=lang, geocode=geocode, since=str(yesterday)).items(200):
    try: 
        print('{} ({}{}): \n{}'.format(t.user.name, t.user.location, get_lat_lon(t.user.location), t.text))
        tweet_latlon.append(get_lat_lon(t.user.location))
#         print(api.get_status(t.id).geo) # dir(t)
    except:
        continue

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    """Distance between 2 geographic locations by lat-lon coordinates."""
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    mi = miles_constant * c
    return mi

In [ ]:
tweet_latlon[:5]

In [ ]:
df_tweets = pd.DataFrame(tweet_latlon, columns=['lat', 'lng', 'accuracy'])
df_tweets.head()

In [ ]:
latlon

In [ ]:
df_tweets['distance'] = haversine(latlon['lat'], latlon['lng'], df_tweets['lat'].values, df_tweets['lng'].values)

In [ ]:
df_tweets.head()

#### Create bounding box

Convert radius in kilometers to degrees.

In [ ]:
area = radius * 10
zoom_level = int(20/np.log(area)//1 + 4)

lat_deg = area/110.574
lon_deg = area/(111.320 * np.cos(latlon['lat']))
bbox = [latlon['lng']-lon_deg, latlon['lng']+lon_deg, latlon['lat']-lat_deg, latlon['lat']+lat_deg]
bbox, zoom_level

In [ ]:
import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM # maps

osm_tiles = OSM()

plt.figure(figsize=(16, 16))

# Use the tile's projection for the underlying map.
ax = plt.axes(projection=osm_tiles.crs)

# Specify a region of interest, in this case, Cardiff.
ax.set_extent(bbox, ccrs.PlateCarree())

# Add the tiles at zoom level
# zoom level determines the detail level of the bbox AND 
# the combination of radius and zoom level determines the loading time(9 ~ too coarse, 11 ~ several minutes)
# zoomlevel 10: country level - area 20
# zoomlevel 9: country level - area 100
# zoomlevel 8: country level - area 200
# zoomlevel 7: country level - area 300
# zoom_level = int(20/np.log(x)//1 + 4)
ax.add_image(osm_tiles, zoom_level)

# Add tweets to plot
plt.plot(df_tweets.loc[df_tweets['distance'] <= radius, 'lng'], 
         df_tweets.loc[df_tweets['distance'] <= radius, 'lat'], 
         transform=ccrs.PlateCarree(),
         marker='o', 
         color='blue', 
         markersize=12, 
         alpha=.4,
         linestyle='')

plt.plot(df_tweets.loc[df_tweets['distance'] > radius, 'lng'], 
         df_tweets.loc[df_tweets['distance'] > radius, 'lat'], 
         transform=ccrs.PlateCarree(),
         marker='o', 
         color='red', 
         markersize=12, 
         alpha=.4,
         linestyle='')

# ax.coastlines('10m')
plt.axis('off')
plt.show();

#### read our own timeline (i.e. our Twitter homepage)

In [ ]:
for status in tweepy.Cursor(api.home_timeline).items(10):
    # Process a single status
    print(status.text)

The function process_or_store() is a place-holder for your custom implementation. In the simplest form, you could just print out the JSON, one tweet per line:

In [ ]:
def process_or_store(tweet):
    return json.dumps(tweet)

#### JSON format

In [ ]:
for status in tweepy.Cursor(api.home_timeline).items(10):
    # Process a single status
    status._json['text']

#### List of all your followers

In [ ]:
for friend in tweepy.Cursor(api.friends).items(10):
    friend._json['screen_name']

#### List of all your tweets

In [ ]:
for tweet in tweepy.Cursor(api.user_timeline).items(10):
    tweet._json['text']

In this way we can easily collect tweets (and more) and store them in the original JSON format,  
fairly easy to convert into different data models depending on our storage (many NoSQL technologies provide some bulk import feature).

### Streaming

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open('python.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True

    
# twitter_stream = Stream(auth, MyListener())
# twitter_stream.filter(track=['#python'])

In [ ]:
# !head python.json

In [ ]:
import json
 
with open('python.json', 'r') as f:
    line = f.readline() # read only the first tweet/line
    tweet = json.loads(line) # load it as Python dict
    print(json.dumps(tweet, indent=4)) # pretty-print

#### Tokenize tweet

In [ ]:
from nltk.tokenize import word_tokenize
 
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
print(word_tokenize(tweet))

#### 

In [ ]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
print(preprocess(tweet))


In [ ]:
with open('python.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tokens = preprocess(tweet['text'])
        print(tokens)

#### Word frequency

In [ ]:
import operator 
import json
from collections import Counter
 
fname = 'python.json'
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_all = [term for term in preprocess(tweet['text'])]
        # Update the counter
        count_all.update(terms_all)
    # Print the first 5 most frequent words
    print(count_all.most_common(5))

In [ ]:
from nltk.corpus import stopwords
import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']
# We can now substitute the variable terms_all in the first example with something like:

terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]

In [ ]:
terms_stop

#### More term filters

In [ ]:
# Count terms only once, equivalent to Document Frequency
terms_single = set(terms_all)
# Count hashtags only
terms_hash = [term for term in preprocess(tweet['text']) 
              if term.startswith('#')]
# Count terms only (no hashtags, no mentions)
terms_only = [term for term in preprocess(tweet['text']) 
              if term not in stop and
              not term.startswith(('#', '@'))] 
              # mind the ((double brackets))
              # startswith() takes a tuple (not a list) if 
              # we pass a list of inputs


In [ ]:
terms_only

In [ ]:
from nltk import bigrams 
 
terms_bigram = bigrams(terms_stop)

#### Create Yahoo! Where On World ID dictionary

The Yahoo! Where On Earth ID for the entire world is 1.  
[Find your WOE ID](http://woeid.rosselliot.co.nz)

In [ ]:
def get_woeid(locations):
    import requests
    from bs4 import BeautifulSoup
    woeids = {}
    if type(locations) != type([]): 
        locations = [locations]
    for loc in locations:
        try:
            response = requests.get('http://woeid.rosselliot.co.nz/lookup/' + loc)
            results_page = BeautifulSoup(response.content, 'lxml')
            woeids[loc] = int(results_page.find_all('td', {"class": "woeid"})[0].text)
        except:
            continue
    return woeids

In [ ]:
LOCATIONS = ['World', 'United States', 'Netherlands', 'Europe', 'Amsterdam', 'Abcoude']
WOEID = get_woeid(LOCATIONS)

In [ ]:
for loc in LOCATIONS:
    print(loc, WOEID[loc])

### Trends by tweet volume

In [ ]:
def woe_trends_(location):
    """Returns localised trends"""
    WOEID = get_woeid(location)
    local_trends = twitter_api.trends.place(_id=WOEID[location])[0]['trends']
    return (pd.DataFrame([(trend['name'],trend['tweet_volume']) 
                         for trend in local_trends], 
                         columns=['trend', 'volume'])
                         .sort_values('volume', ascending=False)
                         .set_index('trend'))

In [ ]:
woe_trends_('Amsterdam').head(10)

### Locality of trends

In [ ]:
def trending_location(location):
    """Returns localised trends"""
    WOE_ID = get_woeid(location)
    local_trends = twitter_api.trends.place(_id=WOE_ID[location])[0]['trends']
    return set([trend['name'] for trend in local_trends])

In [ ]:
def trending_shared(locations):
    """Returns trends shared between WOE_ID_1 & WOE_ID_2"""
    WOE_ID = get_woeid(locations)
    local_trends_1 = twitter_api.trends.place(_id=WOE_ID[locations[0]])[0]['trends']
    local_trends_2 = twitter_api.trends.place(_id=WOE_ID[locations[1]])[0]['trends']
    set_1 = set([trend['name'] for trend in local_trends_1])
    set_2 = set([trend['name'] for trend in local_trends_2])
    return list(set_1 & set_2)

In [ ]:
def trending_excl(locations):
    """Returns trends only in WOE_ID_1"""
    WOE_ID = get_woeid(locations)
    local_trends_1 = twitter_api.trends.place(_id=WOE_ID[locations[0]])[0]['trends']
    local_trends_2 = twitter_api.trends.place(_id=WOE_ID[locations[1]])[0]['trends']
    set_1 = set([trend['name'] for trend in local_trends_1])
    set_2 = set([trend['name'] for trend in local_trends_2])
    return list((set_1 ^ set_2) & set_1)

In [ ]:
trends_set = {}
trends_set['world'] = trending_location('World')
trends_set['nl'] = trending_location('Netherlands')
trends_set['amsterdam'] = trending_location('Amsterdam')
trends_set['nl&ams'] = trending_shared(['Netherlands', 'Amsterdam'])
trends_set['ams^nl'] = trending_excl(['Amsterdam', 'Netherlands'])

In [ ]:
trends_set['ams^nl']
trends_set['ams^nl'][0]
trends_set['nl&ams'][0]

In [ ]:
trending_shared(['Netherlands', 'Germany'])

### Collecting search results

Set the variable `q` to a trending topic, 
or anything else for that matter. The example query below
was a trending topic when this content was being developed
and is used throughout the remainder of this chapter

[api docs](https://dev.twitter.com/docs/api/1.1/get/search/tweets)

In [ ]:
def recursive_nodes(x, **kwargs):
    """"""
    key = kwargs.get('key', None)
    y = kwargs.get('y', None)
    
    # base case
    if y is None: y = []
    if x == []: return y
    
    # recursive call
    if key is None: 
        y.append(x[0])
#         recursive_nodes(x[1:], None, y)
    else: 
        y.append(x[0][key])
    recursive_nodes(x[1:], key=key, y=y)
    return y

In [ ]:
assert recursive_nodes([1,2,3,4]) == [1, 2, 3, 4]

In [ ]:
def trending_topic(topic, number=100):
    """Returns status"""
    return twitter_api.search.tweets(q=topic, count=number)['statuses']

In [ ]:
def trending_topic(topic, number=100):
    """Returns status"""
    return twitter_api.search.tweets(q=topic, count=number)['statuses']

In [ ]:
def trending_text(topic, number=100, head=10):
    """Returns status text"""
    tweet_ = [(s['user']['screen_name'], recursive_nodes(s['entities']['user_mentions'], key='screen_name'),
               recursive_nodes(s['entities']['hashtags'], key='text'), s['text'], s['retweet_count']) 
              for s in trending_topic(topic)]
    return pd.DataFrame(tweet_, columns=['name', 'mentions', 'hashtags', 'text', 'retweets']).sort_values('retweets', ascending=False)

In [ ]:
topic = trends_set['ams^nl'][0]
statuses = trending_topic(topic)
statuses[0].keys()

In [ ]:
statuses[0]#['geo']

In [ ]:
df = trending_text(topic, number=100, head=10)
df.head(100)

In [ ]:
df.mentions[:10]
df.hashtags[:10]

### Extracting text, screen names, and hashtags from tweets

In [ ]:
# status_texts = [status['text'] for status in statuses]

# screen_names = [user_mention['screen_name'] for status in statuses
#                                             for user_mention in status['entities']['user_mentions']]

# hashtags = [hashtag['text'].lower() for status in statuses
#                             for hashtag in status['entities']['hashtags']]
stopwords = ['de', 'het', 'een', 'is', 'de', 'die', 'dat', 'dit', 'van', 'en', 'rt', 'in', 'er', 'op', 'als', 'aan', 'als', 'bij',
             'met', 'niet', 'voor', 'gaat', 'ze', 'je', 'ik', 'wij', 'rt', 'staan', 'kan', 'dan', 'af', 'zoals', 'laat', 'naar',
             'meer', 'werd', 'geen', 'na', 'heeft', 'komt', 'wel', 'nog', 'over', '-']

# Compute a collection of all words from all tweets
words = [w.lower() for t in df['text']
           for w in t.split() if w.lower() not in stopwords]
Counter(words).most_common(20)

In [ ]:
status_texts[0:5]

In [ ]:
# Explore the first 5 items for each...
print('status text: ', json.dumps(status_texts[0:5], indent=1))
print('screen names: ', json.dumps(screen_names[0:5], indent=1)) 
print('hashtags: ', json.dumps(hashtags[0:5], indent=1))
print('words: ', json.dumps(words[0:5], indent=1))

### Basic frequency distribution from the words in tweets

In [ ]:
from collections import Counter

for item in words: #[words, screen_names, hashtags]:
    c = Counter(item)
    print('-'*80)
    print(c.most_common()[:10]) # top 10
    

In [ ]:
pd.DataFrame(Counter(words).most_common(30), columns=['word', 'count']).set_index('word').head()

In [ ]:
pd.DataFrame(Counter(screen_names).most_common(30), columns=['mentions', 'count']).set_index('mentions').head()

In [ ]:
pd.DataFrame(Counter(hashtags).most_common(30), columns=['hashtags', 'count']).set_index('hashtags').head()

### Most popular retweets

In [ ]:
retweets = [
            # Store out a tuple of these three values ...
            (status['retweet_count'], 
             status['retweeted_status']['user']['screen_name'],
             status['text'].replace("\n","\\")) 
            
            # ... for each status ...
            for status in statuses 
            
            # ... so long as the status meets this condition.
                if 'retweeted_status' in status
           ]

In [ ]:
df_retweets = pd.DataFrame(retweets, columns=['retweets', 'screen_name', 'text']).sort_values('retweets', ascending=False)

In [ ]:
df_retweets.head()

In [ ]:
df_retweets.text[16]